# Lab 8: Summarizing the *SARS-COV-2* genome

In this lab we will use Biopython to summarize the *Severe acute respiratory syndrome coronavirus 2 (SARS-Cov-2)* genome.

The entry has been downloaded from https://www.ncbi.nlm.nih.gov/nuccore/NC_045512 and saved as a text file which is available on the course page.

In [1]:
from Bio import SeqIO # to parse sequence data

with open('SARS-COV-2.gbk') as handle :
    sequences = SeqIO.parse(handle, "genbank")
    seq_record = next(sequences) # get the first sequence record

seq_record

SeqRecord(seq=Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...AAA', IUPACAmbiguousDNA()), id='NC_045512.2', name='NC_045512', description='Wuhan seafood market pneumonia virus isolate Wuhan-Hu-1, complete genome', dbxrefs=['BioProject:PRJNA485481'])

### Find all protein products from CDS features

For each CDS, output the gene name, protein id, and product in a table format that uses *f-strings* (see below) so that each column has a width of 20 characters. 

In [2]:
print(f'{"GENE":20}{"Protein ID":20}{"Product":20}')

for feature in seq_record.features:
    if feature.type == 'CDS':
        gene = feature.qualifiers['gene'][0]
        protein_id = feature.qualifiers['protein_id'][0]
        product = feature.qualifiers['product'][0]
        print(f'{gene:20}{protein_id:20}{product:20}')

GENE                Protein ID          Product             
orf1ab              YP_009724389.1      orf1ab polyprotein  
orf1ab              YP_009725295.1      orf1a polyprotein   
S                   YP_009724390.1      surface glycoprotein
ORF3a               YP_009724391.1      ORF3a protein       
E                   YP_009724392.1      envelope protein    
M                   YP_009724393.1      membrane glycoprotein
ORF6                YP_009724394.1      ORF6 protein        
ORF7a               YP_009724395.1      ORF7a protein       
ORF7b               YP_009725296.1      ORF7b               
ORF8                YP_009724396.1      ORF8 protein        
N                   YP_009724397.2      nucleocapsid phosphoprotein
ORF10               YP_009725255.1      ORF10 protein       


### Find all mature protein products of *orf1ab*

The gene *orf1ab* is known as a *polyprotein*. A polyprotein is a gene that is translated into a single protein that is then cleaved (cut) into multiple proteins, which are then referred to as *mature peptides*. Output all of the mature peptides produced by *orf1ab*. Also use Python to count and then output the number of mature peptides produced by *orf1ab*.

In [6]:
count = 0
print(f'{"GENE":20}{"Protein ID":20}{"Product":20}')
print()

for feature in seq_record.features:
    if feature.type == 'mat_peptide':
        if feature.qualifiers['gene'][0] == 'orf1ab':
            count += 1
            gene = feature.qualifiers['gene'][0]
            protein_id = feature.qualifiers['protein_id'][0]
            product = feature.qualifiers['product'][0]
            print(f'{gene:20}{protein_id:20}{product:20}')
print()
print(f'Number of mature peptides from orf1ab: {count}')

GENE                Protein ID          Product             
orf1ab              YP_009725297.1      leader protein      
orf1ab              YP_009725298.1      nsp2                
orf1ab              YP_009725299.1      nsp3                
orf1ab              YP_009725300.1      nsp4                
orf1ab              YP_009725301.1      3C-like proteinase  
orf1ab              YP_009725302.1      nsp6                
orf1ab              YP_009725303.1      nsp7                
orf1ab              YP_009725304.1      nsp8                
orf1ab              YP_009725305.1      nsp9                
orf1ab              YP_009725306.1      nsp10               
orf1ab              YP_009725307.1      RNA-dependent RNA polymerase
orf1ab              YP_009725308.1      helicase            
orf1ab              YP_009725309.1      3'-to-5' exonuclease
orf1ab              YP_009725310.1      endoRNAse           
orf1ab              YP_009725311.1      2'-O-ribose methyltransferase
orf1ab 

### Extracting viral RNA sequences for coronavirus testing

Current tests for *SARS-COV-2* involve testing for the presence of viral RNA molecules. For example, "the Stanford test screens first for the presence of viral RNA encoding a protein called an envelope protein, which is found in the membrane that surrounds the virus and plays an important role in the viral life cycle, including budding from an infected host cells. It then confirms the positive result by testing for a gene encoding a second protein called RNA-dependent RNA polymerase."

Source: http://med.stanford.edu/news/all-news/2020/03/stanford-medicine-COVID-19-test-now-in-use.html

**Question**: Output the RNA sequences for the *envelope protein* CDS and the *RNA-dependent RNA polymerase*, which is a *mature protein*. Note: you should output the gene sequences from the seq_record entry; these sequences are actually DNA sequences -- the viral RNA is identical but will have uracil instead of thymine.


In [24]:
for feature in seq_record.features:
    if feature.type == 'CDS' or feature.type == 'mat_peptide':
        sequence = seq_record.seq
        product = feature.qualifiers['product'][0]
        gene = feature.qualifiers['gene'][0]
        if 'RNA-dependent RNA polymerase' in product:
            print('======================RNA-DEPENDENT RNA POLYMERASE=========================')
            feature_locations = feature.location.parts
            for location in feature_locations:
                print(seq_record.seq[location.start:location.end].transcribe())
                print()
        elif 'envelope protein' in product:
            print('============================ENVELOPE PROTEIN==============================')
            feature_locations = feature.location
            print(seq_record.seq[feature_locations.start:feature_locations.end].transcribe())
            print()

======================RNA-DEPENDENT RNA POLYMERASE=========================
UCAGCUGAUGCACAAUCGUUUUUAAAC

CGGGUUUGCGGUGUAAGUGCAGCCCGUCUUACACCGUGCGGCACAGGCACUAGUACUGAUGUCGUAUACAGGGCUUUUGACAUCUACAAUGAUAAAGUAGCUGGUUUUGCUAAAUUCCUAAAAACUAAUUGUUGUCGCUUCCAAGAAAAGGACGAAGAUGACAAUUUAAUUGAUUCUUACUUUGUAGUUAAGAGACACACUUUCUCUAACUACCAACAUGAAGAAACAAUUUAUAAUUUACUUAAGGAUUGUCCAGCUGUUGCUAAACAUGACUUCUUUAAGUUUAGAAUAGACGGUGACAUGGUACCACAUAUAUCACGUCAACGUCUUACUAAAUACACAAUGGCAGACCUCGUCUAUGCUUUAAGGCAUUUUGAUGAAGGUAAUUGUGACACAUUAAAAGAAAUACUUGUCACAUACAAUUGUUGUGAUGAUGAUUAUUUCAAUAAAAAGGACUGGUAUGAUUUUGUAGAAAACCCAGAUAUAUUACGCGUAUACGCCAACUUAGGUGAACGUGUACGCCAAGCUUUGUUAAAAACAGUACAAUUCUGUGAUGCCAUGCGAAAUGCUGGUAUUGUUGGUGUACUGACAUUAGAUAAUCAAGAUCUCAAUGGUAACUGGUAUGAUUUCGGUGAUUUCAUACAAACCACGCCAGGUAGUGGAGUUCCUGUUGUAGAUUCUUAUUAUUCAUUGUUAAUGCCUAUAUUAACCUUGACCAGGGCUUUAACUGCAGAGUCACAUGUUGACACUGACUUAACAAAGCCUUACAUUAAGUGGGAUUUGUUAAAAUAUGACUUCACGGAAGAGAGGUUAAAACUCUUUGACCGUUAUUUUAAAUAUUGGGAUCAGACAUACCACCCAAAUUGUGUUAACUGUUUGGAUGACAGAUGCAUUC

### Aside: f-strings

An *f-string* is a formatted string in Python. Formatting includes specification of the width of the output, or specifying the number of decimal places for a decimal number. We will only worry about setting the output width here.

The basic syntax of a formatted string is
```
f'text {value:format} more text'
```

where *value* is a string or the name of a variable, and format can be

- a number, to denote the width (e.g., 10) (with default left-alignment)
- a number, preceded by a '<' for left-alignment, a '^' for center-alignment, or a '>'for right-alignment

The cell below demonstrates how to display 'hi' inside a block of 10 characters, where 'hi' is left-aligned, centered, or right-aligned.

In [ ]:
print(f'{"hi":<10}') # left-align
print(f'{"hi":^10}') # center-align
print(f'{"hi":>10}') # right-align

We can also use *f-strings* to display the values of variables, as in the example below.

In [ ]:
fName = 'Jane'
lName = 'Doe'
print(f'Hello {fName} {lName}!')